# <center> **John Wick EDA** </center>

![jwick.jpg](https://wallpaperaccess.com/full/983569.jpg)

# Introduction
In this notebook, I will be analyzing one of the most iconic neo-noir action-thriller media franchise in the 20th Century, John Wick!  The entire franchise started with John Wick, a retired legendary hitman, searching for the men who broke into his home, stole his vintage car and killed his puppy, which was his deceased wife's last gift to him. 

In [169]:
#Data Imports
#Load Libraries
library(tidyverse)
library(dplyr)
library(janitor)
library(cowplot)
library(sysfonts)
library(showtext)

In [170]:
#Load Dataset into a dataframe
jwick <- read_csv("../input/john-wicks-confirmed-onscreen-kills/John Wick.csv", show_col_types = FALSE) %>%
clean_names()

In [171]:
#Add Fonts
font_add_google("Revalia")
font_add_google("Teko")
font_add_google("Anton")
font_add_google("Roboto")

#Load Fonts
font1 <- "Revalia"
font2 <- "Teko"
font3 <- "Anton"
font4 <- "Roboto"

#Show external text effects
showtext_auto()

In [172]:
#Data Exploration and Data Wrangling
glimpse(jwick)

In [173]:
head(jwick) #snapshot of first 6 rows of the dataframe

In [174]:
#Check the number of films recorded
distinct(jwick['movie'])

In [175]:
#Replace names for cohesiveness and easier reading.
jwick$movie<- replace(jwick$movie, jwick$movie == 'John Wick', 'Chapter 1') 
jwick$movie<- replace(jwick$movie, jwick$movie == 'John Wick Chapter 2', 'Chapter 2') 
jwick$movie<- replace(jwick$movie, jwick$movie == 'John Wick: Chapter 3 – Parabellum', 'Chapter 3')

In [176]:
#Check for confirmation
distinct(jwick['movie'])

# Does John Wick have a vendetta towards the Russians?
Throughout the franchise, the Russians play a central role to the story due to being portrayed as the main antagonists. Film after film, fight after fight, it seems like John Wick can't get enough of the Russians. Is this inevitable or vengeance? To answer this, let's see how many Russians have been killed relative to the total kills.

In [177]:
distinct(jwick['who'])

In [178]:
#Create new dataframe
killtypes <- jwick %>%
drop_na(who, kill_id) %>%
group_by(who)  %>%
summarise(kill_id = n()) %>%
arrange(desc(kill_id)) 

View(killtypes)

In [179]:
nrow(subset(killtypes, kill_id ==1))

In [180]:
#Remove rows
killtypes <- killtypes[-c(7,8,9,10,11,12,13,14,15),]

# Combined all antagonists into one row.
mafia <- c("Antagonist", 9)
killtype_v2 <- rbind(killtypes, mafia)
killtype_v2$kill_id <- as.integer(killtype_v2$kill_id)

percent <- killtype_v2 %>%
mutate(kill_id = as.integer(kill_id)) %>%
transmute(percentage = (kill_id/sum(kill_id))*100,
      percentage = round(percentage, digits = 2))

killtype_v3 <- cbind(killtype_v2, percent)

killtype_v3

In [181]:
killtype_v3$ymax = cumsum(killtype_v3$percentage)
killtype_v3$ymin = c(0, head(killtype_v3$ymax, n = -1))

#Compute label position and label
killtype_v3$label_position <- (killtype_v3$ymax + killtype_v3$ymin) / 2
killtype_v3$label <- paste0(killtype_v3$who, "\n", killtype_v3$percentage, '%')

In [182]:
#Donut Chart
options(repr.plot.width = 10, repr.plot.height = 10) #Chart Dimensions
donut <- 
killtype_v3 %>%
ggplot(aes(ymax = ymax, ymin = ymin, xmax = 3, xmin = 1, fill = who)) +
geom_rect() +
geom_label(x=2.25, aes(y=label_position, label = label), size = 3.25) +
scale_fill_brewer(palette = 'Reds') +
coord_polar(theta = 'y') +
xlim(c(-1,4)) +
theme_void() +
theme(legend.position = 'none',
     plot.background = element_rect(fill = '#d6d6d4'))

#Finaized Donut Chart:
finalized_donut <- 
ggdraw(donut) +
draw_label("KILLS By AFFILIATION", fontfamily = font2, x = 0.5, y = 0.5, color = '#363533', size = 75) +
draw_label("Creation: Brandon Qilin Hung", fontfamily = font4, x = 0.5, y =0.1, color = 'black', size = 15)

#Show Graph
finalized_donut
ggsave("affiliationkills.png", device = png, height=10, width=10, dpi=100)

**Analysis**

Throughout the franchise, 23.34% and 4.88% of John Wick's kills are Russian Mobster and Russian Hit Crew. That means 28.22% of John Wick's kills are Russians. To my surprise, this only ranks it as the second highest. The highest kill per affiliation are Bodyguards, a whopping 36.59%! As expected, antagonists only makes up 3.14%. The antagonists includes Ares, Avi, Ernest, D'Antonio, Iosef TaraSov, Viggo TaraSov, Kirill, and Zero.

# What type of weapon racks up the most kills for John Wick?
John Wick's favorite weapon is the Heckler & Koch P30L fitted with a custom compensator. This can be seen numerous times as he would always resort to it as his primary firearm. Does his favorite pistol rack up the most kills compared to his weapon arsenal?

In [183]:
#Search and find weapons John Wick utilizes
distinct(jwick['weapon'])
count(distinct(jwick['weapon']))

Unequivocally and obvious at first sight, John Wick is a master weapon user as he is able to wield 16 different types of weapons!

In [184]:
#Create dataframe
weapontypes <- jwick %>%
select(kill_id, weapon) %>%
drop_na(kill_id) %>%
group_by(weapon) %>%
summarise(kill_id = n()) %>%
 arrange(desc(kill_id))

weapontypes$weapontype <- c("Firearm", "Firearm", "Firearm", "Melee", "Melee", "Vehicle", "Melee", "Firearm", "Firearm", "Uncommon", "Explosives", "Uncommon", "Melee", "Uncommon", "Uncommon")

weapontypes

**Explanation:** 


Of course, John Wick's favorite go-to firearm, the hangun, has the highest net kills, 163. But hey, who would have expected that John Wick had 3 horse kills?

In [185]:
#Download Images
#Source: 
#Deagle-https://www.artstation.com/artwork/28ZEGa
download.file("https://i.imgur.com/kUbjMmR.png", "brokenglass.png", mode ='wb')
download.file("https://i.imgur.com/J0Yv8vA.png", "deagle.png", mode = 'wb')


#Load Images
shattered <- "brokenglass.png"
deagle <- "deagle.png"


In [186]:
options(repr.plot.width = 20, replr.plot.height = 20)

#Color Palette
colorpalette <- c("#8c898a", "#e62249", "#ffccdd", "#ff8b6e", "#c46d02")

#Circular Barplot
circularplot <- 
weapontypes %>%
ggplot(aes(x=as.factor(weapon), y=as.integer(kill_id) +15, fill = weapontype)) +
       geom_bar(stat = 'identity') +
scale_fill_manual(values = colorpalette) +
coord_polar(start = 0) +
ylim(c(-25,200)) +
theme_void() +
labs(fill = 'Weapon Type') +
theme(plot.title = element_text(hjust = 0.5, vjust =-5, color = 'white', size = 50),
    plot.background = element_rect(fill = 'black'),
     legend.title = element_text(color = 'white', face = 'bold', size = 20),
     legend.text = element_text(size = 12.5, color = 'white'),
     legend.position = c(0.925,0.525),
     legend.key.size = unit(1, 'cm'))

#Finalized Circular Barplot
final_circ <-
ggdraw(circularplot) +
draw_image(shattered, x= 0.05, y=0.0375, scale = 0.45) +
#draw_image(deagle, x = -0.1, y= -0.4, scale = 0.2) +
draw_label(label = 'KILLS\n By \n WEAPON TYPE', fontfamily = font2, x = 0.5, y= 0.825, size = 75, color ='white') +
draw_label(label = '163', x = 0.565, y= 0.2, size = 40, color = 'white', fontfamily = font1, angle = 22.5) +
draw_label(label = 'Weapon Kill Record: \n #1. Handgun \n #2. Assault Rile \n #3. Shotgun \n | \n v \n #15. Book', fontfamily = font3, color = 'white', x = 0.175, y =0.575, size = 25, alpha = 0.6) +
draw_label(label = 'Creation: Brandon Qilin Hung', x= 0.8825, y = 0.15, fontfamily = font4, size = 20, color ='white')

#Show Graph
final_circ
ggsave("killsperweapon.png", device = png, height = 20, width = 20, dpi = 100)

**Analaysis**

As expected, the hangun racks up the most kills with a total of 163! Per weapon type, the firearm is the category with the highest number of kills. Although the quantity differs, the number of kills in explosives, uncommon, and vehicle are quite even across the board.

# How many people did John Wick killed over the course of 3 films?
To shoot bullet in the coffin, I analyzed the total number of kills John Wick racked up til the latest film, Chapter 3: Parabellum. Despite the seemingly endless bloodbath and pew-pew, how many people were actually killed from the hands of John Wick?

In [187]:
count(jwick['kill_id']) #Total enemies faced by John Wick
count(drop_na(jwick['kill_id'])) #Total enemies killed by John Wick

**Explanation** 

There is a total of 547 values in kill_id, which includes null values. Excluding the null values, there is only 287 values in kill_id. This means that John Wick's current kill record is 287.

In [188]:
#Data Preparation and Data Wrangling
killcount <- 
jwick %>% 
drop_na(kill_id)

In [189]:
#Check for confirmation
glimpse(killcount)
distinct(killcount['movie'])
is.null(jwick['kill_id'])

In [190]:
#Check the total number of kills based on film.
killcount %>% group_by(movie) %>% summarise(kill_id = n())

In [191]:
#Data Visualization
#Set Image Dimensions
options(repr.plot.width = 40, repr.plot.height = 20)

#Kill Count Plot
kc <- 
killcount %>%
group_by(movie) %>%
summarise(kill_id = n()) %>%
drop_na(movie) %>%
ggplot() +
geom_col(aes(x=movie, y=kill_id, fill = '#ff0000')) +
coord_flip() +
theme_void() +
theme(plot.background = element_rect(fill = 'black', color = 'black'),
     plot.margin = margin(10,10,40,10),
     legend.position = 'none') 

In [192]:
#Load Images
download.file("https://i.imgur.com/ehFn0GR.png", "jwick.png", mode = 'wb')
download.file("https://i.imgur.com/8iSIawk.png", "used_bullets.png", mode = 'wb')
wick <- "jwick.png"
bullets <- "used_bullets.png"

#Finalized Kill Count Plot 
kc_final <-
ggdraw(kc) +
draw_image(bullets, x = 0.3, y =-.325, scale = 0.4) +
draw_image(wick, x = -.295, y = 0.035, scale = 0.85) +
draw_label(label = 'CONFIRMED KILLS', x = 0.2, y= 0.5, fontfamily = font2, color = '#ffd6d6', fontface = 'bold', size = 175, angle = 30) +
draw_label(label = '76', x = 0.635, y = 0.89, fontfamily = font1, color = 'white', size = 250) +
draw_label(label = 'Chapter 1', x = 0.56, y = .75, fontfamily = font3, color = 'black', size = 250) +
draw_label(label = '123', x = 0.88, y = 0.585, fontfamily = font1, color = 'white', size = 250) +
draw_label(label = 'Chapter 2', x = 0.81, y = 0.45, fontfamily = font3, color = 'black', size = 250) +
draw_label(label = '88', x = 0.545, y = 0.285, fontfamily = font1, color = 'white', size = 250) +
draw_label(label = 'Chapter 3', x = 0.467, y= 0.15, fontfamily = font3, color = 'black', size = 250) +
draw_label(label = 'Creation: Brandon Qilin Hung', x = 0.9, y =0.035, fontfamily = font4, color = 'white', size = 50)

#Show Graph
kc_final
ggsave("totalkills.png", device = png, height = 40, width = 20, dpi = 200)

**Analysis**

Over the course of three films, John Wick has killed 287 people. The highest number of kills Wick netted is in the sequel film, Chapter 2. The lowest number of kills is 76 in the first film, Chapter 1. That leaves Chapter 3 as the Median for number of kills, which is 88.

# <center> ~FIN </center>